In [101]:
DATA_DIR = '/home/nsuprotivniy/Documents/Работа/OKru/Antispam/data/'
GRAPH_PATH = '/home/nsuprotivniy/Documents/Работа/OKru/Antispam/graph/'

SAMPLE_FRAC = 0.01 


FEATURES = [
    'DeviceClass',
    'DeviceName',
    'DeviceBrand',
    'DeviceCpu',
    'DeviceCpuBits',
    'OperatingSystemClass',
    'OperatingSystemName',
    'OperatingSystemVersion',
    'OperatingSystemNameVersion',
    'OperatingSystemVersionBuild',
    'LayoutEngineClass',
    'LayoutEngineName',
    'LayoutEngineVersion',
    'LayoutEngineVersionMajor',
    'LayoutEngineNameVersion',
    'LayoutEngineNameVersionMajor',
    'AgentClass',
    'AgentName',
    'AgentVersion',
    'AgentVersionMajor',
    'AgentNameVersion',
    'AgentNameVersionMajor',
#     'from',
#     'to',
#     'url',
#     'requestType',
    'operation'
]

In [102]:
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa

from sklearn.preprocessing import OneHotEncoder
from sklearn.base import TransformerMixin

%matplotlib inline
from matplotlib import pyplot as plt

pd.set_option("display.precision", 2)

### Модели преобразования данных

In [103]:
class HashingTrick(TransformerMixin):
    
    _default_hashing_trick_modulars = {
        'DeviceClass': 10000,
        'DeviceName': 10000,
        'DeviceBrand': 10000,
        'DeviceCpu': 10000,
        'DeviceCpuBits': 10000,
        'OperatingSystemClass': 10000,
        'OperatingSystemName': 10000,
        'OperatingSystemVersion': 10000,
        'OperatingSystemNameVersion': 10000,
        'OperatingSystemVersionBuild': 10000,
        'LayoutEngineClass': 10000,
        'LayoutEngineName': 10000,
        'LayoutEngineVersion': 10000,
        'LayoutEngineVersionMajor': 10000,
        'LayoutEngineNameVersion': 10000,
        'LayoutEngineNameVersionMajor': 10000,
        'AgentClass': 10000,
        'AgentName': 10000,
        'AgentVersion': 10000,
        'AgentVersionMajor': 10000,
        'AgentNameVersion': 10000,
        'AgentNameVersionMajor': 10000,
        'from': 10000,
        'to': 10000,
        'url': 10000000,
        'requestType': 10000,
        'operation': 10000
    }
    
    def __init__(self, hashing_trick_modulars = _default_hashing_trick_modulars):
        self.hashing_trick_modulars = hashing_trick_modulars
    
    def set_params(self, **kwargs):
        """Set the parameters of this estimator."""
    
    def get_params(self, **kwargs):
        return {"hashing_trick_modulars": self.hashing_trick_modulars}
        
    def _hashing_trick(self, x, n):
        return hash(x) % n

    def _column_hashing_trick(self, col_name):
        self.http[col_name] = self.http[col_name].apply(self._hashing_trick, args=(self.hashing_trick_modulars[col_name],))
  
    def fit_transform(self, X, *_):
        return self.transform(X)
        
    def transform(self, X, *_):
        self.http = X
        for feature in FEATURES:
            self._column_hashing_trick(feature)

        return self.http
    
    def fit(self, *_):
        return self

In [104]:
ht = HashingTrick()
ohe = OneHotEncoder(handle_unknown='ignore')

### Загрузка данных из паркета

In [105]:
X = pq.read_table(DATA_DIR + 'HTTPRequests-20180217_1718_parsedUAA.parquet', columns=FEATURES ).to_pandas().sample(frac=SAMPLE_FRAC)
y = pq.read_table(DATA_DIR + 'HTTPRequests-20180217_1718_parsedUAA.parquet', columns=['isBot'] ).to_pandas().sample(frac=SAMPLE_FRAC)

### Преобразование данных

In [106]:
X = ht.transform(X)
X = ohe.fit_transform(X)

### Преобразует номер фичи в её имя

In [112]:
def feature_name(i):
    a_from = ohe.feature_indices_[:ohe.feature_indices_.shape[0] - 1]
    a_to = ohe.feature_indices_[1:]
    for i_from, i_to,k in zip(a_from, a_to, range(0, a_from.shape[0])):
        if (i in range(i_from, i_to)):
            return FEATURES[k]
    return 'unknown'

In [114]:
feature_name(7), feature_name(43242), feature_name(543254325432543254325), feature_name(-1)

('DeviceClass', 'DeviceCpuBits', 'unknown', 'unknown')